In [ ]:
#Prompt Engineering with LLM GOOGLE FLAN T5 Model 
#Instance type used on sagemaker - ml.x5.2xlarge
#Dataset used - Dialogue Sum Hugging Face Dataset


In [51]:

%pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch==1.13.1 \
    torchdata==0.5.1 --quiet

%pip install \
    transformers==4.27.2 \
    datasets==2.11.0  --quiet

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.
DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.
DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version 

Load the datasets, Large Language Model (LLM), tokenizer, and configurator. Do not worry if you do not understand yet all of those components - they will be described and discussed later in the notebook.

In [4]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

In [52]:
#Using simple dialogues from the DialogSum Hugging Face dataset. This dataset contains 10,000+ dialogues with the corresponding manually labeled summaries 

huggingface_dataset_name = "knkarthick/dialogsum"
dataset = load_dataset(huggingface_dataset_name)
print(dataset["train"][40])
print(dataset["test"][40])

Found cached dataset csv (/root/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-3005b557c2c04c1d/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


  0%|          | 0/3 [00:00<?, ?it/s]

{'id': 'train_40', 'dialogue': "#Person1#: I just bought a new dress. What do you think of it?\n#Person2#: You look really great in it. So are you going to a job interview or a party?\n#Person1#: No, I was invited to give a talk in my school.\n#Person2#: So how much did you pay for it?\n#Person1#: I pay just $70 for it. I saved $30.\n#Person2#: That's really a bargain.\n#Person1#: You're right. Well, what did you do while I was out shopping?\n#Person2#: I watched TV for a while and then I did some reading. It wasn't a very interesting book so I just read a few pages. Then I took a shower.\n#Person1#: I thought you said you were going to see Mike.\n#Person2#: I'll go and visit him at his home tomorrow. He'll return home tomorrow morning.\n#Person1#: I'm glad he can finally returned home after that accident.", 'summary': 'While #Person1# made a bargain to buy a new dress, #Person2# watched TV, read a boring book, and took a shower at home.', 'topic': 'go shopping'}
{'id': 'test_13_2', 'd

In [53]:
example_indices = [40, 200]

dash_line = '-'.join('' for x in range(100))


for index, val in enumerate(example_indices):
    print(index,val)
    print(dash_line)
    print('Example ', index + 1)
    print(dash_line)
    print('Input Dialogue:',dataset['test'][val]['dialogue'])
    print(dash_line)
    print('Human Summary:',dataset['test'][val]['summary'])
    print(dash_line)
   

0 40
---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
Input Dialogue: #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
Human Summary: #Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
1 200
---------------------------------------------------------------------------------------------------
Ex

In [54]:
#loading the GOOGLE FLAN-T5 LLM Model
model_name='google/flan-t5-base'
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [55]:
#Loading Tokenizer for the model
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

In [56]:
sentence = "What time is it, Tom?"

sentence_encoded = tokenizer(sentence, return_tensors='pt')
print("sentence after encoding",sentence_encoded)
print("tensoor mask of the encoded sentence",sentence_encoded["input_ids"][0])

sentence_decoded = tokenizer.decode(
        sentence_encoded["input_ids"][0], 
        skip_special_tokens=True
    )
print('\nDecoded Sentence:',sentence_decoded)


sentence after encoding {'input_ids': tensor([[ 363,   97,   19,   34,    6, 3059,   58,    1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}
tensoor mask of the encoded sentence tensor([ 363,   97,   19,   34,    6, 3059,   58,    1])

Decoded Sentence: What time is it, Tom?


In [36]:
for i, index in enumerate(example_indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']
    
    inputs = tokenizer(dialogue, return_tensors='pt')
    generated_text = model.generate(inputs["input_ids"], max_new_tokens=50)# generates a list of tensors for a numebr of sentences where each sentence represents a tensor
    print("generated text tensor",generated_text) 
    output = tokenizer.decode(generated_text[0],  #decode the generated text
        skip_special_tokens=True
    )
    # generating text without prompt engineering
    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print(f'Input Prompt:\n{dialogue}')
    print(dash_line)
    print(f'Human Summary:\n{summary}')
    print(dash_line)
    print(f'Model generating output without using Prompt Engineering:\n{output}\n')

generated text tensor tensor([[   0, 5780,  536,   10,   94,   31,    7,    3,  324,   12, 4169,    5,
            1]])
---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
--------------------------------------------------------------------------------------

In [57]:
# zero shot inference with an instruction prompt
for i, index in enumerate(example_indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    prompt = f"""
Summarize the following conversation.

{dialogue}

Summary:
    """

    print(f'INPUT PROMPT:\n{prompt}')
    # Input constructed prompt instead of the dialogue.
    inputs = tokenizer(prompt, return_tensors='pt')
    generated_text = model.generate(inputs["input_ids"],max_new_tokens=50)
    output = tokenizer.decode(generated_text[0],skip_special_tokens=True)
    
    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print(dash_line)
    print(f'Human Summary:\n{summary}')
    print(dash_line)    
    print(f'Summary generated by the model using zero shot inference:\n{output}\n')

INPUT PROMPT:

Summarize the following conversation.

#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

Summary:
    
---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
---------------------------------------------------------------------------------------------------
Human Summary:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
Summary generated by the model using zero shot inference:
Th

In [58]:
#Zero Shot Inference with the Prompt Template from FLAN-T5
for i, index in enumerate(example_indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']
        
    prompt = f"""
Dialogue:

{dialogue}

What was going on?
"""

    
    print(f'INPUT PROMPT:\n{prompt}')
    inputs = tokenizer(prompt, return_tensors='pt')
    generated_text = model.generate(inputs["input_ids"],max_new_tokens=50)
    output = tokenizer.decode(generated_text[0],skip_special_tokens=True)

    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
    print(dash_line)
    print(f'MODEL GENERATION - ZERO SHOT:\n{output}\n')

INPUT PROMPT:

Dialogue:

#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

What was going on?

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT:
Tom is late for the train.

INPUT PROMPT:


In [42]:
def make_prompt(example_indices_full, example_index_to_summarize): # do not exceed tokens by 512
    prompt = ''
    for index in example_indices_full:
        dialogue = dataset['test'][index]['dialogue']
        summary = dataset['test'][index]['summary']
        
        # The stop sequence '{summary}\n\n\n' is important for FLAN-T5. Other models may have their own preferred stop sequence.
        prompt += f"""
Dialogue:

{dialogue}

What was going on?
{summary}


"""
    
    dialogue = dataset['test'][example_index_to_summarize]['dialogue']
    
    prompt += f"""
Dialogue:

{dialogue}

What was going on?
"""
        
    return prompt

In [59]:
# generating prompt for one shot inference
num_examples = [40] 
example_to_ask_for_summary = 200
one_shot_prompt = make_prompt(num_examples, example_to_ask_for_summary)
print(one_shot_prompt)


Dialogue:

#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

What was going on?
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.



Dialogue:

#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also ne

In [60]:
# generating model summary using one shot prompt engineering
summary = dataset['test'][example_index_to_summarize]['summary']

inputs = tokenizer(one_shot_prompt, return_tensors='pt')
generated_text = model.generate(inputs["input_ids"],max_new_tokens=50)
output = tokenizer.decode(generated_text[0],skip_special_tokens=True)
print(dash_line)
print(f'Example of Human Summary given:\n{summary}\n')
print(dash_line)
print(f'Summary generated by the model using one prompt:\n{output}')

---------------------------------------------------------------------------------------------------
Example of Human Summary given:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.

---------------------------------------------------------------------------------------------------
Summary generated by the model using one prompt:
#Person1 wants to upgrade his system. #Person2 wants to add a painting program to his software. #Person1 wants to add a CD-ROM drive.


In [61]:
# generating prompt for few shot inference

num_examples = [40, 80, 120]
example_to_ask_for_summary = 200
few_shot_prompt = make_prompt(num_examples, example_to_ask_for_summary)
print(few_shot_prompt)


Dialogue:

#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

What was going on?
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.



Dialogue:

#Person1#: May, do you mind helping me prepare for the picnic?
#Person2#: Sure. Have you checked the weather report?
#Person1#: Yes. It says it will be sunny all day. No sign of rain at all. This is your father's favorite sausage. Sandwiches for you and Daniel.
#Person2#: No, thanks Mom. I'd like some toast and chicken wings.
#Person1#: Okay. Please take some fruit salad and crackers for me.
#Person2#: Done. Oh, don't forget to take napkins disposable plates, cups and picnic blanket.
#Person1#: All set. 

In [63]:
# model generating summary using few shot prompts

summary = dataset['test'][example_index_to_summarize]['summary']
inputs = tokenizer(few_shot_prompt, return_tensors='pt')
generated_text = model.generate(inputs["input_ids"],max_new_tokens=50)
output = tokenizer.decode(generated_text[0],skip_special_tokens=True)
print(dash_line)
print(f'Given human summary:\n{summary}\n')
print(dash_line)
print(f'Summary generated by the model using few prompts:\n{output}')

---------------------------------------------------------------------------------------------------
Given human summary:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.

---------------------------------------------------------------------------------------------------
Summary generated by the model using few prompts:
#Person1 wants to upgrade his system. #Person2 wants to add a painting program to his software. #Person1 wants to upgrade his hardware.
